In [1]:
import sys
sys.path.append('../../../../')

In [2]:
from dataclasses import dataclass

import datetime
import pprint
import itertools
import requests
import pandas as pd
import typing

from utils.subgraph_utils.constants import SUBGRAPH_API

## Get pool data containing coins, pool addresses, coin decimals

In [3]:
def get_pool_data(api):

    query = """
    {
        platforms {
            pools(first: 1000) {
                coins
                coinDecimals
                address
            }
        }
    }
    """
    r = requests.post(api, json={"query": query})
    data = dict(r.json())
    pool_data = data["data"]["platforms"][0]["pools"]

    return pool_data

In [4]:
network_name = 'Mainnet'

In [5]:
data = get_pool_data(SUBGRAPH_API[network_name])
data[:2]

[{'coins': ['0xa713cc74ee148414bcab46ac2c41c93d84a56b0f',
   '0x6c3f90f043a72fa612cbac8115ee7e52bde6e490'],
  'coinDecimals': ['18', '18'],
  'address': '0x0043fcb34e7470130fde28198571dee092c70bd7'},
 {'coins': ['0x15a629f0665a3eb97d7ae9a7ce7abf73aeb79415',
   '0x9c4a4204b79dd291d6b6571c5be8bbcd0622f050'],
  'coinDecimals': ['18', '18'],
  'address': '0x01fe650ef2f8e2982295489ae6adc1413bf6011f'}]

## Vet pools
Remove pools with either no liquidity (coins less than a certain amount) or no activity in the past year.

In [6]:
def get_num_swaps_pool(pool_addr, api, activity_duration: int = 365):

    time_end = int(datetime.datetime.now().timestamp())
    time_start = int(datetime.datetime.now().timestamp() - 24*3600*activity_duration)

    query = f"""
    {{
      swapEvents(
        first: 1000,
        where: {{
          pool: "{pool_addr.lower()}"
          timestamp_gte: {time_start}
          timestamp_lt: {time_end}
        }}
      ) {{
        timestamp
        block
      }}
    }}
    """
    r = requests.post(api, json={'query': query})
    queried_data = dict(r.json())
    if not 'data' in queried_data:
      print("no data")
      return 0
    swap_events = queried_data['data']['swapEvents']
    return len(swap_events)

In [7]:
def get_latest_pool_coin_reserves(pool_addr, api):

    query = f"""
    {{
      dailyPoolSnapshots(
        first: 1,
        orderBy: timestamp,
        orderDirection: desc,
        where:{{
          pool: "{pool_addr.lower()}"
        }}
      ) {{
        reserves
      }}
    }}
    """
    r = requests.post(api, json={'query': query})
    queried_data = dict(r.json())['data']['dailyPoolSnapshots'][0]['reserves']
    return [int(i) for i in queried_data]

In [8]:
print(f"total pools in the graph: {len(data)}")
vetted_pools = []
num_days_to_look_back_for_swap_events = 365
reserve_threshold = 100  # num coins of each type in the pool
for pool_data in data:

    pool_address = pool_data['address']
    coin_decimals = [int(i) for i in pool_data['coinDecimals']]
    latest_coin_reserves = get_latest_pool_coin_reserves(pool_addr=pool_address, api=SUBGRAPH_API[network_name])
    pool_reserve_critera_met = all(
        [reserves > reserve_threshold*10**coin_decimals[idx] for idx, reserves in enumerate(latest_coin_reserves)]
    )

    if not pool_reserve_critera_met:
        # print(f"pool: {pool_address} didn't make the cut with reserves: {latest_coin_reserves}")
        continue

    vetted_pools.append(pool_data)

vetted_pools[0]

total pools in the graph: 295


{'coins': ['0x15a629f0665a3eb97d7ae9a7ce7abf73aeb79415',
  '0x9c4a4204b79dd291d6b6571c5be8bbcd0622f050'],
 'coinDecimals': ['18', '18'],
 'address': '0x01fe650ef2f8e2982295489ae6adc1413bf6011f'}

In [9]:
print(f"all pools: {len(data)}")
print(f"vetted pools: {len(vetted_pools)}")

all pools: 295
vetted pools: 134


## Generate coin maps that show routes between two coins

In [10]:
class Coin(typing.NamedTuple):
    """A dataclass cacheing some coin info and a few basic methods."""
    address: str
    network: str
    decimals: int


class BasePool(typing.NamedTuple):
    """A dataclass cacheing basepool lp token and coins in the base pool"""
    pool_address: str
    lp_token: Coin
    lp_token_decimals: int
    network: str
    coins: typing.List[Coin]


@dataclass(eq=True, frozen=True)
class Pool:
    """A dataclass containing details on the pool connecting two assets"""
    address: str
    network: str
    coin_a: Coin
    coin_b: Coin


@dataclass
class Route:
    """A dataclass containing multi hops between coin a and coin b"""
    n_hops: int
    pools: typing.List[Pool]
    coin_a: str
    coin_b: str


In [11]:
class CoinMap:

    def __init__(self, coins: typing.List[str], base_pools: typing.List[BasePool]):

        self.number_of_coins = len(coins)
        self.coins = coins
        self.coin_pairs = {coin: set() for coin in coins}
        self.coin_pair_pool = {}
        self.base_pools = base_pools

    def add_pair(self, coin_a: Coin, coin_b: Coin, pool: Pool):

        if coin_a not in self.coin_pairs.keys():
            self.coin_pairs[coin_a] = set()

        self.coin_pairs[coin_a].add((coin_b, pool))
        self.coin_pair_pool[(coin_a, coin_b)] = pool

    def get_route(self, coin_a: Coin, coin_b: Coin, max_hops: int = 5) -> typing.List[Route]:

        print(f"Pathfinding for coins between {coin_a.address} -> {coin_b.address}")

        # hack to reduce graph size: usdt, usdc and dai are just threecrv in this search space
        for base_pool_data in self.base_pools:
            if coin_a in base_pool_data.coins:
                coin_a = base_pool_data.lp_token
                print(f"Coin in basepool. Searching for coins between {coin_a} -> {coin_b} instead.")

            if coin_b in base_pool_data.coins:
                coin_b = base_pool_data.lp_token
                print(f"Coin in basepool. Searching for coins between {coin_a} -> {coin_b} instead.")

        coin_paths = self._depth_first_search(coin_a, coin_b)

        # construct the swap route for coin pair
        all_coin_routes = []
        for coin_path in coin_paths:
            coin_pairs_in_path = list(zip(coin_path, coin_path[1:])) 
            constructed_swap_route = [
                self.coin_pair_pool[coin_pair] for coin_pair in coin_pairs_in_path
            ]

            if len(constructed_swap_route) > max_hops:
                continue

            coin_route = Route(coin_a=coin_a, coin_b=coin_b, n_hops=len(constructed_swap_route), pools=constructed_swap_route)

            if not coin_route in all_coin_routes:
                all_coin_routes.append(coin_route)

        return all_coin_routes

    def _depth_first_search(
        self, coin_to_sell: Coin, target_coin_to_buy: Coin, path: typing.List = [], max_hops: int = 5
    ):

        path = path + [coin_to_sell]

        if coin_to_sell == target_coin_to_buy:
            return [path]

        if coin_to_sell not in self.coin_pairs.keys():
            return []

        if len(path) > max_hops:
            return []

        paths = []
        for (coin, pool) in self.coin_pairs[coin_to_sell]:
            if coin not in path:
                paths.extend(self._depth_first_search(coin, target_coin_to_buy, path))

        return paths

    def print_route(self, coin_a: Coin, coin_b: Coin, max_hops: int = 5):

        all_routes = coin_map.get_route(coin_a, coin_b, max_hops)
        
        c = 1
        for route in all_routes:
            print(f"Route {c}")
            c += 1
            for hops in route.pools:
                print(f"Coin in: {hops.coin_a.address} -> Pool: {hops.address} -> Coin out: {hops.coin_b.address}\n")
            print("\n")
                

## Get all pairs

In [12]:
all_coins_in_vetted_pools = []
for pool in vetted_pools:

    coins_in_pool = []    
    for idx, coin in enumerate(pool['coins']):
        coin_dataclass = Coin(address=coin, network=network_name, decimals=int(pool['coinDecimals'][idx]))
        all_coins_in_vetted_pools.append(coin_dataclass)

all_coins_in_vetted_pools = list(set(all_coins_in_vetted_pools))
all_coins_in_vetted_pools[0]

Coin(address='0x1c48f86ae57291f7686349f12601910bd8d470bb', network='Mainnet', decimals=18)

In [13]:
usdt = Coin("0xdAC17F958D2ee523a2206206994597C13D831ec7".lower(), network='Mainnet', decimals=6)
usdc = Coin("0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48".lower(), network='Mainnet', decimals=18)
dai = Coin("0x6B175474E89094C44Da98b954EedeAC495271d0F".lower(), network='Mainnet', decimals=18)
fei = Coin(address="0x956F47F50A910163D8BF957Cf5846D573E7f87CA".lower(), network='Mainnet', decimals=18)
cvxcrv = Coin("0x62B9c7356A2Dc64a1969e19C23e4f579F9810Aa7".lower(), network='Mainnet', decimals=18)
crv = Coin("0xD533a949740bb3306d119CC777fa900bA034cd52".lower(), network='Mainnet', decimals=18)
threecrv_token = Coin("0x6c3f90f043a72fa612cbac8115ee7e52bde6e490", 'Mainnet', 18)

threecrv = BasePool(
    pool_address="0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7".lower(),
    lp_token=threecrv_token, 
    network="Mainnet", 
    lp_token_decimals=18, 
    coins=[dai, usdc, usdt]
)


In [14]:
coin_map = CoinMap(all_coins_in_vetted_pools, base_pools=[threecrv])

# add weth <-> eth as the first pair
weth_address = "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2".lower()
eth_address = "0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE".lower()
weth = Coin(address=weth_address, network='Mainnet', decimals=18)
eth = Coin(address=eth_address, network="Mainnet", decimals=18)

weth_eth_pool = Pool(address=weth_address, network="Mainnet", coin_a=weth, coin_b=eth)
eth_weth_pool = Pool(address=eth_address, network="Mainnet", coin_a=eth, coin_b=weth)

coin_map.add_pair(weth, eth, weth_eth_pool)
coin_map.add_pair(eth, weth, eth_weth_pool)

all_pairs = []
for pool in vetted_pools:
    pool_address = pool['address']
    coins_in_pool = []    
    for idx, coin in enumerate(pool['coins']):
        coin_dataclass = Coin(address=coin, network=network_name, decimals=int(pool['coinDecimals'][idx]))
        coins_in_pool.append(coin_dataclass)

    coin_permutations = list(itertools.permutations(coins_in_pool))
    for coin_permutation in coin_permutations:
        coin_pair_pool = Pool(address=pool_address, network=network_name, coin_a=coin_permutation[0], coin_b=coin_permutation[1])
        all_pairs.append(coin_pair_pool)
        coin_map.add_pair(coin_permutation[0], coin_permutation[1], coin_pair_pool)

In [15]:
coin_map.base_pools[0]

BasePool(pool_address='0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7', lp_token=Coin(address='0x6c3f90f043a72fa612cbac8115ee7e52bde6e490', network='Mainnet', decimals=18), lp_token_decimals=18, network='Mainnet', coins=[Coin(address='0x6b175474e89094c44da98b954eedeac495271d0f', network='Mainnet', decimals=18), Coin(address='0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', network='Mainnet', decimals=18), Coin(address='0xdac17f958d2ee523a2206206994597c13d831ec7', network='Mainnet', decimals=6)])

## Get all routes for a coin

In [16]:
coin_map.print_route(fei, usdt)

Pathfinding for coins between 0x956f47f50a910163d8bf957cf5846d573e7f87ca -> 0xdac17f958d2ee523a2206206994597c13d831ec7
Coin in basepool. Searching for coins between Coin(address='0x956f47f50a910163d8bf957cf5846d573e7f87ca', network='Mainnet', decimals=18) -> Coin(address='0x6c3f90f043a72fa612cbac8115ee7e52bde6e490', network='Mainnet', decimals=18) instead.
Route 1
Coin in: 0x956f47f50a910163d8bf957cf5846d573e7f87ca -> Pool: 0x06cb22615ba53e60d67bf6c341a0fd5e718e1655 -> Coin out: 0x6c3f90f043a72fa612cbac8115ee7e52bde6e490



Route 2
Coin in: 0x956f47f50a910163d8bf957cf5846d573e7f87ca -> Pool: 0xbaaa1f5dba42c3389bdbc2c9d2de134f5cd0dc89 -> Coin out: 0x853d955acef822db058eb8505911ed77f175b99e

Coin in: 0x853d955acef822db058eb8505911ed77f175b99e -> Pool: 0x4e0915c88bc70750d68c481540f081fefaf22273 -> Coin out: 0xa693b19d2931d498c5b318df961919bb4aee87a5

Coin in: 0xa693b19d2931d498c5b318df961919bb4aee87a5 -> Pool: 0xceaf7747579696a2f0bb206a14210e3c9e6fb269 -> Coin out: 0x6c3f90f043a72fa612cba

Check if route from FEI -> cvxCRV is possible.

It should be via: 

1. FEI -> USDT 
2. USDT -> WETH 
3. WETH -> ETH (unwrap)
4. ETH -> CRV 
5. CRV -> cvxCRV

In [17]:
coin_map.print_route(fei, cvxcrv, 5)

Pathfinding for coins between 0x956f47f50a910163d8bf957cf5846d573e7f87ca -> 0x62b9c7356a2dc64a1969e19c23e4f579f9810aa7
Route 1
Coin in: 0x956f47f50a910163d8bf957cf5846d573e7f87ca -> Pool: 0xbaaa1f5dba42c3389bdbc2c9d2de134f5cd0dc89 -> Coin out: 0x853d955acef822db058eb8505911ed77f175b99e

Coin in: 0x853d955acef822db058eb8505911ed77f175b99e -> Pool: 0x4e0915c88bc70750d68c481540f081fefaf22273 -> Coin out: 0xdac17f958d2ee523a2206206994597c13d831ec7

Coin in: 0xdac17f958d2ee523a2206206994597c13d831ec7 -> Pool: 0xd51a44d3fae010294c616388b506acda1bfaae46 -> Coin out: 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2

Coin in: 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 -> Pool: 0x8301ae4fc9c624d1d396cbdaa1ed877821d7c511 -> Coin out: 0xd533a949740bb3306d119cc777fa900ba034cd52

Coin in: 0xd533a949740bb3306d119cc777fa900ba034cd52 -> Pool: 0x9d0464996170c6b9e75eed71c68b99ddedf279e8 -> Coin out: 0x62b9c7356a2dc64a1969e19c23e4f579f9810aa7





In [18]:
coin_map.print_route(crv, cvxcrv)

Pathfinding for coins between 0xd533a949740bb3306d119cc777fa900ba034cd52 -> 0x62b9c7356a2dc64a1969e19c23e4f579f9810aa7
Route 1
Coin in: 0xd533a949740bb3306d119cc777fa900ba034cd52 -> Pool: 0x9d0464996170c6b9e75eed71c68b99ddedf279e8 -> Coin out: 0x62b9c7356a2dc64a1969e19c23e4f579f9810aa7





In [19]:
coin_map.print_route(usdt, fei)

Pathfinding for coins between 0xdac17f958d2ee523a2206206994597c13d831ec7 -> 0x956f47f50a910163d8bf957cf5846d573e7f87ca
Coin in basepool. Searching for coins between Coin(address='0x6c3f90f043a72fa612cbac8115ee7e52bde6e490', network='Mainnet', decimals=18) -> Coin(address='0x956f47f50a910163d8bf957cf5846d573e7f87ca', network='Mainnet', decimals=18) instead.
Route 1
Coin in: 0x6c3f90f043a72fa612cbac8115ee7e52bde6e490 -> Pool: 0x06cb22615ba53e60d67bf6c341a0fd5e718e1655 -> Coin out: 0x956f47f50a910163d8bf957cf5846d573e7f87ca



Route 2
Coin in: 0x6c3f90f043a72fa612cbac8115ee7e52bde6e490 -> Pool: 0xd632f22692fac7611d2aa1c0d552930d43caed3b -> Coin out: 0x853d955acef822db058eb8505911ed77f175b99e

Coin in: 0x853d955acef822db058eb8505911ed77f175b99e -> Pool: 0xbaaa1f5dba42c3389bdbc2c9d2de134f5cd0dc89 -> Coin out: 0x956f47f50a910163d8bf957cf5846d573e7f87ca



Route 3
Coin in: 0x6c3f90f043a72fa612cbac8115ee7e52bde6e490 -> Pool: 0xd632f22692fac7611d2aa1c0d552930d43caed3b -> Coin out: 0x853d955acef